### Transforming raw data into time series data for our analysis

In our solution we will be predicting the number of rides(drivers needed) per location per hour.

Therefore, I will extract the rides for each hour, then aggregage them to find the number of rides for each location.

In [1]:
import pandas as pd # load pands library

rides = pd.read_parquet('../data/transformed/validated_rides_2022_01.parquet') # load transformed rides data

rides.head() # show first 5 rows of data

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [2]:
# adding new column 'pickup_hour' that etracts the hour from the 'pickup_datetime' column
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


In [3]:
# now lets group by pickup_hour and pickup_location_id so as to get the number of rides per hour per location.
# here I'm counting the number of rows of which each row represents a ride
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index() # with size() we get the number of rows per group
agg_rides.rename(columns={0: 'rides'}, inplace=True) # renaming the column with the number of rides
agg_rides

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,4,11
1,2022-01-01 00:00:00,7,6
2,2022-01-01 00:00:00,10,1
3,2022-01-01 00:00:00,12,2
4,2022-01-01 00:00:00,13,12
...,...,...,...
66863,2022-01-31 23:00:00,261,4
66864,2022-01-31 23:00:00,262,8
66865,2022-01-31 23:00:00,263,26
66866,2022-01-31 23:00:00,264,24


**Observations and insights**

You can now observe that we have the number of rides per hour per location.

Our target variable will be the number of rides. `We want to predict the number of rides per hour per location`.

In the groupby method, in the event that there are no rides at a given location and time, this can result into incomplete timeseries data due to missing points. Let's address this!

In [4]:
# Let us build a function to add missing points
from tqdm import tqdm # load tqdm library to show progress bar

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame: # function to add missing slots
    
    location_ids = agg_rides['pickup_location_id'].unique() #extract a list of unique location ids
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H') # create a list of all hours between the min and max pickup_hour and then buils a series hour by hour
    output = pd.DataFrame()
    for location_id in tqdm(location_ids): # for each location id

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'rides']]
            
        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)
        
        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id

        output = pd.concat([output, agg_rides_i]) # concatenate the output with the agg_rides_i
    
    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'})
    
    return output

In [5]:
agg_rides_all_slots = add_missing_slots(agg_rides)

  0%|          | 0/257 [00:00<?, ?it/s]

100%|██████████| 257/257 [00:00<00:00, 446.00it/s]


In [6]:
# Now let us plot the time series data
from typing import Optional, List 
import plotly.express as px # load plotly library

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

In [7]:
plot_rides(agg_rides_all_slots, locations=[43])

**Observations and insights**

As expected the data from pickup_location_id 43 is seasonal.

We can see that the data has been transformed into time series data.

Let us save our time series data for future use

In [8]:
agg_rides_all_slots.to_parquet('../data/transformed/ts_data_2022_01.parquet')